In [1]:
from pprint import pprint
import re
import json
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import sys
import codecs
from elasticsearch import Elasticsearch, ElasticsearchException, SerializationError,TransportError,\
ConnectionError, ConnectionTimeout, NotFoundError, RequestError
from elasticsearch.helpers import bulk, BulkIndexError, parallel_bulk
from bs4 import BeautifulSoup
from collections import deque
import pysftp
import os
from datetime import datetime
sys.path.append('/home/michaelp/REVIEWS')
import util
logfilename="log.txt"
logfile=open(logfilename, 'a')
logfile.write("established log file: "+datetime.now().isoformat()+"\n")
logfile.flush()
INDEX_NAME = "publication"  # the name of the index
DOC_TYPE = "html"  # we have a single type of document, so it doesn't matter
REMOVE_STOPWORDS=True
STEM=False
INDEX_SETTINGS = {  # two shards with a single replica
    "settings" : {
        "index" : {
            "number_of_shards" : 6,
            "number_of_replicas" : 1
           #,"similarity" : { "default" : { "type" : "LMJelinekMercer", "lambda":"0.4" }
            }
        }
        ,
    "mappings": {
            "html": {
            "properties": {
#                "post_date": { "type": "date" }
#                ,
                "title": {"type": "text" }
                ,
#                "id": { "type": "keyword"}
#                ,
                "text": { "type": "text" }
                ,
                "description": {"type": "text"}
                ,
                "keywords": {"type": "keyword"}
                ,
                "classification":{"type": "text"}
                }
            }
        }
    }

DCTR=0
DOCS_IN_FILE=0
es = Elasticsearch("localhost:9211", timeout=100)
if es.indices.exists(INDEX_NAME):
    print("index exists")
    #es.indices.delete(index=INDEX_NAME)
else:
    print("cannot")
    sys.exit()
#pprint (es.indices.stats(index="publication"))


'''
body = {
	
    "size": 1000,           # get 100 docs (default = 10)
    "_source": ["work.subjects"], # get only wanted fields
	 "query": { 
         "query_string":{"query":"Oslo"}# the query
	 }
}
result=es.search(index="publication", body=body)
'''
result = es.search(index=INDEX_NAME, q="Oslo", _source=False, size=1000)
type(result)
pprint(result)



full_url: http://localhost:9211/publication
index exists
full_url: http://localhost:9211/publication/_search?_source=false&q=Oslo&size=1000
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp9fa88bf03f89c56afe5c986157bb1852',
                    '_index': 'publication',
                    '_score': 11.59626,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fpf579492e328e4f22ee845b6003015e7b',
                    '_index': 'publication',
                    '_score': 11.59626,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fpefa12493b8490d07e435a2a4ff88fe0e',
                    '_index': 'publication',
                    '_score': 11.59626,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fpa1efb4c01837c2d6

                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp8d89c3c0545b0c81b7c501d5d537a6b2',
                    '_index': 'publication',
                    '_score': 8.756474,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp8759cc2112c89edd5f158643151c8a57',
                    '_index': 'publication',
                    '_score': 8.756474,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp354037850920',
                    '_index': 'publication',
                    '_score': 8.756474,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp9d4ba8e16d571f7fc48c997c27fd10d0',
                    '_index': 'publication',
                    '_score': 8.756474,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp81d507d025a61

                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2F94f88822-536c-4380-b930-f1513ef634ac',
                    '_index': 'publication',
                    '_score': 8.011732,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp699021954515',
                    '_index': 'publication',
                    '_score': 8.011732,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp902460218168',
                    '_index': 'publication',
                    '_score': 8.011732,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fpb62c0cdbf6b14e5b46db9c7e9f243404',
                    '_index': 'publication',
                    '_score': 8.011732,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp1977c3a3283298434a9c8383261321

                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp531513030833',
                    '_index': 'publication',
                    '_score': 6.957531,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp616951678686',
                    '_index': 'publication',
                    '_score': 6.957531,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp243168036935',
                    '_index': 'publication',
                    '_score': 6.957531,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2F6b467fcd-fb25-44b7-aa09-75d2f3ca2700',
                    '_index': 'publication',
                    '_score': 6.957531,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp703940046350',
                    '_index': 'publ

                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp3a1d0885ff874c5ffa7cb1eb21ba3684',
                    '_index': 'publication',
                    '_score': 6.4500885,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fpdd9365369b54ed487c3c70a2247a93bb',
                    '_index': 'publication',
                    '_score': 6.4500885,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp3227e3cd40465a066b2ceebab385ffb0',
                    '_index': 'publication',
                    '_score': 6.4500885,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp4aa4782dea64284aeb37d4264257c843',
                    '_index': 'publication',
                    '_score': 6.4500885,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpubl

                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp4fe96c7f8e368975e78096bda3882bb2',
                    '_index': 'publication',
                    '_score': 6.4500885,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp0b031cd541131d0f01867b894ae47cbb',
                    '_index': 'publication',
                    '_score': 6.416824,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp8cda53a757c6f464b3ecb59503842e53',
                    '_index': 'publication',
                    '_score': 6.413348,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp8cf5893ab5c874f752403af27d3301df',
                    '_index': 'publication',
                    '_score': 6.413348,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublica

                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp546c5b6ee40602fca69543e7a1da800d',
                    '_index': 'publication',
                    '_score': 6.413348,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp5c2c9708d81868e412274a23c852568a',
                    '_index': 'publication',
                    '_score': 6.413348,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp656c95417d956320d71d514ca0e6d404',
                    '_index': 'publication',
                    '_score': 6.413348,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp6950d5d0bc96901cde569396f6c85818',
                    '_index': 'publication',
                    '_score': 6.413348,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublicat

                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fpf17530e3987b3450f706907382d220e2',
                    '_index': 'publication',
                    '_score': 6.3701735,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp86d424b9cd615e12065f4532b432bba7',
                    '_index': 'publication',
                    '_score': 6.310406,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp06bd7fe81e891c921f4db0d916dccc41',
                    '_index': 'publication',
                    '_score': 6.2866497,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp73458c856d64eaca3527e9cb8c4dcbce',
                    '_index': 'publication',
                    '_score': 6.2803345,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpubli

                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fpa5d2800b943b6e67c4c3d3cabc76e534',
                    '_index': 'publication',
                    '_score': 6.1835,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp2f3e5207c8a4e84a5c1af79c945ecee9',
                    '_index': 'publication',
                    '_score': 6.1835,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp3f2006e9fb427c104b64f7cc8feb1c87',
                    '_index': 'publication',
                    '_score': 6.1835,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp6d9b46a425aca0c461ba64b7863cd194',
                    '_index': 'publication',
                    '_score': 6.1835,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp9

                    '_index': 'publication',
                    '_score': 6.025777,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp64a5dbecc04c072c664a07f49d618560',
                    '_index': 'publication',
                    '_score': 6.025777,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp26c1b1eac52a7abd3229b244462834fe',
                    '_index': 'publication',
                    '_score': 6.025777,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp5a184c64ccb1ddc38ead470269dc60e5',
                    '_index': 'publication',
                    '_score': 6.025777,
                    '_type': '_doc'},
                   {'_id': 'http%3A%2F%2Fdata.deichman.no%2Fpublication%2Fp639b598f5808ce003ba54e0a46722cb7',
                    '_index': 'publication',
                    '_s